In [47]:
PYTHONIOENCODING='utf-8'
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib, sys, os
import re
import time
import codecs
import json
import traceback
import datetime
import pickle as pk
chromedriver = "./chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [48]:
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',}

In [49]:
opener = urllib.request.build_opener()

In [50]:
def get_year_info():
    req = urllib.request.Request('https://www.aclweb.org/anthology/sigs/sigdat/', headers=hdr)
    response = opener.open(req)
    bsObj = BeautifulSoup(response, "html")
    rows = bsObj.find_all('div', { "class" : "row" })
    all_volume = []
    for row in rows:
        year = row.h4.text
        procedding = row.a.text
        url = 'https://www.aclweb.org' + row.a['href']
        if int(year) > 2009:
            continue
        all_volume.append([int(year), procedding, url])
    return all_volume

In [51]:
all_volumes = get_year_info()

In [52]:
all_volumes = sorted(all_volumes)

In [42]:
all_volumes

[[1993,
  'VERY LARGE CORPORA: ACADEMIC AND INDUSTRIAL PERSPECTIVES',
  'https://www.aclweb.org/anthology/volumes/very-large-corpora-academic-and-industrial-perspectives/'],
 [1994,
  'Second Workshop on Very Large Corpora (WVLC-2)',
  'https://www.aclweb.org'],
 [1995,
  'Third Workshop on Very Large Corpora',
  'https://www.aclweb.org/anthology/volumes/third-workshop-on-very-large-corpora/'],
 [1996,
  'Fourth Workshop on Very Large Corpora',
  'https://www.aclweb.org/anthology/volumes/fourth-workshop-on-very-large-corpora/'],
 [1997,
  'Fifth Workshop on Very Large Corpora',
  'https://www.aclweb.org/anthology/volumes/fifth-workshop-on-very-large-corpora/'],
 [1998,
  'Sixth Workshop on Very Large Corpora',
  'https://www.aclweb.org/anthology/volumes/sixth-workshop-on-very-large-corpora/'],
 [1999,
  '1999 Joint SIGDAT Conference on Empirical Methods in Natural Language Processing and Very Large Corpora',
  'https://www.aclweb.org/anthology/volumes/1999-joint-sigdat-conference-on-em

In [53]:
import time

In [54]:
def get_all_cite(article_url):
    driver.get(article_url)
    atl_page = driver.page_source.encode('utf-8')
    atl_obj = BeautifulSoup(atl_page)
    total_citation = int(atl_obj.find('div', {'class':'gs_ri'}).find('div', {'class':'gs_fl'}).find_all('a')[2].text.split()[-1])
#     cited_section = atl_obj.find('div', {'class':'gs_ri'}).find('div', {'class':'gs_fl'}).find_all('a')[2]
#     total_citation = int(cited_section.text.split()[-1])
#     cited_info_url = 'https://scholar.google.com.sg'+cited_section['href']
    time.sleep(30)
    return total_citation
    
    

In [55]:
def get_volume_info(year_url):
    vlm_req = urllib.request.Request(year_url, headers=hdr)
    vlm_page = opener.open(vlm_req)
    vlm_obj = BeautifulSoup(vlm_page)
    articles = vlm_obj.find('div', {'id': 'main-container'}).section.find_all('p')[2:]
    articles_info = []
    for article in articles:
        atl_url = article.span.a['href']
        title = article.strong.text
        googlescl = 'https://scholar.google.com.sg/scholar?hl=en&as_sdt=0%2C5&q='+title.replace(' ', '+')+'&btnG='
        atl_ctn = get_all_cite(googlescl)
        atl_authors = ', '.join([itm.text for itm in article.find_all('a') if 'anthology/people/' in itm['href']])
        articles_info.append([title, atl_url, atl_authors, atl_ctn])
        print([title, atl_url, atl_authors, atl_ctn])
    return articles_info

In [56]:
all_info = []
for itm in all_volumes:
    print('year', itm[0])
    if itm[0]==1994:
        continue
    cnt=get_volume_info(itm[-1])
    all_info.append([itm[0], cnt])
    with open('%d.pkl'%itm[0], 'wb') as filout:
        pk.dump(cnt, filout)

year 1993


AttributeError: 'NoneType' object has no attribute 'find'